1) 5.5. The array in 5.4 receives a signal at broadside with a field strength of $10{^-5}$ V/m. Add to the model code to find the open circuit signal and noise voltage correlation matrices. Find the SNR  with equal open circuit referenced beamformer weights. From the SNR, find the sensitivity of the receiver.

In [ ]:
########### IMPORTS #############
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as spi

########################################### VARIABLES ############################################
# Define antenna parameters
f = 300e6                 # Frequency in Hz
c = 3e8                 # Speed of light in m/s
lam = c / f             # Wavelength in m
k = 2 * np.pi / lam     # Wave number
eta = 377               # Intrinsic impedance of free space
Pel = 1                 # Radiated power
r = 1                   # Observation distance
l = lam / 2             # Half-wave dipole length
Im = 1                  # Mutual impedance
theta = np.pi/2        # broadside or endfire
phi = np.pi/2           # parallel to the array axis

# Define element spacing range
num_points = 100
d_values = np.array([lam/2])
# Number of elements in the array
N = 4

# system noise
T0 = 290  # Reference temperature in Kelvin
kB = 1.380649e-23  # Boltzmann constant in J/K
BW = 1e6  # Bandwidth in Hz
Tsys = T0*kB*BW  # System noise temperature in Kelvin

# sensitivity
sensitivity = 0

# SNR
SNR = 0

################################################## MAIN ##########################################################
for d in d_values:
    ## Define the receiver positions ##
    rx = np.array([n * d for n in range(N)])    # Positions of elements
    #print("rx \n", rx)                          # debug receiver positions

    ## compute the vector of phase shifted electric fields ##
    E0 = 1j * Im * eta / (2 * np.pi * r)                                          # Amplitude of the E-Field of a dipole on page 84
    E_el = E0 * (np.cos(k*l*np.cos(theta)/2)-np.cos(k*l/2)) / np.sin(theta) # E-field of a dipole
    Ep = np.zeros(N, dtype=complex)                                    # Initialize electric field
    # Ep loop for broadside
    for n in range(N):
        Ep[n] = E_el*np.exp(1j*k*d*np.sin(phi)*np.cos(theta)*(rx[n]-rx[0]))         # Compute electric field
    #print("Ep \n", Ep)                                                     # debug e-field

    ## compute the overlap matrix ##
    A = np.zeros((N, N))                                          # initialize the overlap matrix
    points = 50
    for i in range(N):
        for j in range(N):
            # numerical implementation of the quadrature rule according to equations 4.69 and 4.70
            I = 0                                                 # initialize the integral
            del_phi = 2*np.pi/points                                   # integration weights
            del_theta = del_phi                                   # integration weights
            for m in range(points):
                phi_m = (m - 1/2) * del_phi                       # midpoint for each integration step
                for n in range(int(points/2)):
                    theta_n = (n - 1/2) * del_theta               # midpoint for each integration step

                    # integrand
                    Em = E0 * (np.cos(k*l*np.cos(theta_n)/2)-np.cos(k*l/2)) / np.sin(theta_n) * np.exp(1j*k*d*np.sin(theta_n)*np.cos(phi_m)*rx[i])
                    En = E0 * (np.cos(k*l*np.cos(theta_n)/2)-np.cos(k*l/2)) / np.sin(theta_n) * np.exp(1j*k*d*np.sin(theta_n)*np.cos(phi_m)*rx[j])

                    I += Em * np.conj(En) * np.sin(theta_n) * del_phi * del_theta   # integral approximation
                    #print("I\n", I)                               # debug integral                     
            #print("I \n", I)                                      # debug integral                                 

            A[i, j] = I                                           # assign the value of the integral to the overlap matrix
    
    ## compute the mutual impedance matrix ##
    A = A / (2*eta)
    scalar = 2/np.abs(Im)**2    # Equation 4.108
    Za = scalar * A             # mutual impedance matrix
    #print(Za)                   # debug mutual impedance matrix
          
    ## Compute weights ##
    A_inv = np.linalg.inv(A)    # invert overlap       
    #w = np.dot(A_inv, Ep)      # Compute weights equation 4.86
    w = np.array([1, 1, 1, 1])            # uniform weights for simplicity
    #print("weights \n", w)     # debug weights
    
    ## compute signal response ##
    B = np.zeros((N, N), dtype=complex)     # Initialize signal response
    for m in range(N):
        for n in range(N):
            B[m, n] = Ep[m] * Ep[n].conj()  # Compute signal response equation 4.66
    B = B / (2 * eta)                       # Normalize signal response
    #print("B \n", B)                        # debug signal response

    ## Compute Directivity using equation 4.63 ##
    w_herm = w.conj().T                         # Compute hermitian of weights
    #print("hermitian of weights \n", w_herm)   # debug hermitian of weights
    scalar = 4 * np.pi * (r**2)                 # Compute directivity scalar
    wB = w_herm @ B
    numerator = wB @ w  
    wA = w_herm @ A
    denominator = wA @ w
    D = scalar * numerator / denominator        # Compute directivity equation 4.63
    #print("Directivity \n", D)                 # debug directivity

    # calculate sensitivity
    sensitivity = (10*np.log10(D / Tsys))
    #print(k*d)

    ## Noise Calculations ##
    # calculate signal relationship
    Rs = B
    # calculate noise relationship
    const = 16*kB*T0/np.abs(Im)**2
    Rn = const * B @ A
    numerator = w.conj().T @ Rs @ w
    denominator = w.conj().T @ Rn @ w
    SNR = numerator / denominator

print("Sensitivity (dB/K): ", sensitivity)  # print sensitivity values
print("SNR: ", SNR)  # print SNR values
print("SNR (dB): ", 10 * np.log10(SNR))  # print SNR in dB

/tmp/ipykernel_706/2146866340.py:75: ComplexWarning: Casting complex values to real discards the imaginary part
  A[i, j] = I                                           # assign the value of the integral to the overlap matrix


Sensitivity (dB/K):  (150.35327011069438+0j)


2) For your dipole array receiver code, add a receiver noise model. The front end low noise
amplifiers have noise figure 2 dB, noise resistance RN = 15 Ω, and correlation admittance
Yc = 0. Assume that noise added by components after the LNAs is negligible. Compute the
open circuit loaded receiver noise correlation matrix. Combine the receiver noise and external
noise correlation matrix to find the total noise correlation matrix. For the array and incident
field of the previous assignment, find the SNR with receiver noise included. Compute the
receiver sensitivity G/T from the SNR. How much does the SNR decrease compared to the
result from the earlier assignment without receiver noise? Compare the SNR decrease to the
LNA noise figure.